In [ ]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


In [ ]:
review_filename = 'data/yelp_academic_dataset_review.json'


In [ ]:
import pyspark
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
sc = pyspark.SparkContext().getOrCreate()
spark = SparkSession.builder.getOrCreate()


In [ ]:
raw_review_df = spark.read.json(review_filename)
# raw_review_df.printSchema()
raw_review_df.show()

raw_review_RDD = raw_review_df.rdd
# raw_review_RDD.first().asDict()


In [ ]:
def get_review_tuple(entry):
    """ Parse a line in the review dataset form pyspark.sql.Row to tuple
    Args:
        entry (pyspark.sql.Row): a line in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (str(entry["review_id"]),
            str(entry["user_id"]),
            str(entry["business_id"]),
            int(entry["stars"]),
            int(entry["useful"]),
            int(entry["funny"]),
            int(entry["cool"]),
            str(entry["text"]))


review_RDD = raw_review_RDD.map(get_review_tuple)  # repartition(2)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.take(3)}')
